# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1730]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [1731]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [1732]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [1733]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [1735]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [1702]:
data['income_type'].unique()


array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [1703]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [1704]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [1705]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [1706]:
data.groupby('income_type')['days_employed'].agg(['median'])

,median
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,365213.306266
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [1707]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [1708]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [1709]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [1710]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median() 

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [1711]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [1712]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.**

In [1713]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [1714]:
data.duplicated().sum()

71

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [1715]:
def categorize_income(incom):
    if incom <= 30000:
        return 'E'
    elif incom <= 50000:
        return 'D'
    elif incom <= 200000:
        return 'C'
    elif incom <= 1000000:
        return 'B'
    return 'A'

In [1716]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [1717]:
data['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [1718]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [1719]:
def categorize_purpose(purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'жил' in purpose or 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'

In [1720]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [1721]:
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [1722]:
data.groupby('children')['debt'].agg(['count', 'sum', 'mean'])\
    .sort_values('mean', ascending = False).style.format({'mean' : '{:.2%}'})


,count,sum,mean
children,,,
4,41,4,9.76%
2,2055,194,9.44%
1,4818,444,9.22%
3,330,27,8.18%
0,14149,1063,7.51%
5,9,0,0.00%


In [1723]:
data.pivot_table(index = 'children',  values = 'debt', aggfunc = \
['count', 'sum', 'mean']).sort_values(('mean', 'debt'), ascending = False) \
.style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
children,,,
4,41,4,9.76%
2,2055,194,9.44%
1,4818,444,9.22%
3,330,27,8.18%
0,14149,1063,7.51%
5,9,0,0.00%


**Вывод:** 

1. Скорее всего есть зависимость между фактом наличия детей у заемщиков и возвратом кредита в срок. Так доля просроченных кредитов  заемщиков, у которых нет детей, составляет 0.075438, а у заемщиков, у которых есть дети, доля составляет от 0.081818 до 0.094542 (без учета заемщиков, у которых 4 и 5 детей. Так как размеры выборки у таких заёмщиков небольшие (количество случаев просрочки 4 и 0 соответственно), то эти данные либо не стоит учитывать, либо объединить с группой, где 3 детей).


2. Зависимость между возвратом кредита в срок и количеством детей (1 или 2 ребёнка) явно не прослеживается (0.092346 - 0.094542),а заемщики с 3 детьми более надёжны (0.081818).

<div class="alert alert-block alert-success">✔️
    

__Комментарий от ревьюера №1__

Здорово, что посчитал колличество заемщиков в каждой выборке, и учел неравнозначность выборок. 
    
---
    
Смотри  как эту таблицу можно вывести передав список агрегирующих функций:
    
```python
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})
```
</div>

<div class="alert alert-info"> <b>Комментарии студента:</b> Например, вот так.

</div>



#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [1724]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'total_income_category', 'purpose_category'],
      dtype='object')

In [1725]:
data_pivot = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
data_pivot.columns = ['debt', 'total', '%']
data_pivot = data_pivot.sort_values(by = ('%'), ascending = False)
data_pivot.style.format({('%') : '{:.2%}'})
                

,debt,total,%
family_status,,,
Не женат / не замужем,273,2799,9.75%
гражданский брак,385,4160,9.25%
женат / замужем,927,12302,7.54%
в разводе,84,1189,7.06%
вдовец / вдова,63,952,6.62%


In [1726]:


data_pivot

,debt,total,%
family_status,,,
Не женат / не замужем,273,2799,0.097535
гражданский брак,385,4160,0.092548
женат / замужем,927,12302,0.075354
в разводе,84,1189,0.070648
вдовец / вдова,63,952,0.066176


**Вывод:** 

1. Прослеживается зависимость между семейным положением и возвратом кредита в срок. Так чаще просрочки встречаются в группах: "Не женат / не замужем" и  " гражданский брак" (0.097639 и 0.093130) , реже в группах: "вдовец / вдова", "в разводе" и "женат / замужем" (0.066246 - 0.070648 - 0.075606).

<div class="alert alert-block alert-success">✔️
    

__Комментарий от ревьюера №1__

Верно. по расчетам можно сказать, что заемщики которые состоят или состояли в браке лучше оплачивают кредиты.
</div>

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод:**
1. Количество просроченных кредитов в группах А и С составляет по 2 в каждой. На основании таких небольших выборок не следует делать каких-либо статистических выводов относительно этих групп.


2. В группах B, C и D прослеживается зависимость между уровнем дохода и возвратом кредита в срок. 
- В группе D с доходом от 30001 до 50000 наименьшая частота просрочки (0.060172).
- В группе С с доходом от 50001 до 200000 наибольшая частота просрочки (0.084982).
- С дальнейшим увеличением дохода (группа В с доходом от 200001 до 1000000) частота просрочки кредита снижается (0.070602).

<div class="alert alert-info"> 3. Если сравнить две самые многочисленные категории (С с доходом от 50001 до 200000 и В с доходом от 200001 до 1000000), то можно сделать вывод, что с увеличением дохода частота просрочки кредита снижается.


#### 3.4 Как разные цели кредита влияют на его возврат в срок?

**Вывод:** 

1. Существует зависимость между целями кредита и возвратом его в срок. Наиболее часто просрочка возврата кредита происходит при целях кредита: "операции с автомобилем" и  "получение образования" (0.093480 и 0.092528). Более надежные те заемщики, которые берут кредит на операции с недвижимостью (0.072551), затем следуют заемщики с кредитами на проведение свадьбы (0.079118).

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 

Пропуски в исходных данных могут появится по следующим причинам:
- человеческий фактор. Заполняя таблицы люди могут пропустить какие-то поля;
- особенность данных. Это те данные, которые не обязательны к заполнению (например е-mail или телефон), пользователь может их не оставить. Не всегда у исследователя бывает возможность собрать именно все данные по всем объектам в силу разных причин;
- технические причины - сбой программы.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 

Заполнение медианным значением более надежно, чем заполнение  средним значением, поскольку оно смягчает влияние выбросов. Выбросы данных могут существенно влиять на среднее значение, а на медиану они не влияют.

### Шаг 4: общий вывод.

Мы проверили четыре гипотезы и установили:

1. Существует зависимость между фактом наличия детей у заемщиков и возвратом кредита в срок. Заёмщики без детей имеют минимальную частоту просрочек по кредитам. Наибольшую частоту просрочек по кредитам имеют заёмщики с 1 и 2 детьми. Заёмщики с 3 детьми более надёжны по сравнению с предыдущей группой. 


2. Существует зависимость между семейным положением и возвратом кредита в срок. Чаще просрочки встречаются в группах: "Не женат / не замужем" и " гражданский брак", реже в группах: "вдовец / вдова", "в разводе" и "женат / замужем".


3. Существует зависимость между уровнем дохода и возвратом кредита в срок.
В группе с доходом от 30001 до 50000 наименьшая частота просрочки.
В группе с доходом от 50001 до 200000 наибольшая частота просрочки.
С дальнейшим увеличением дохода (группа  с доходом от 200001 до 1000000) частота просрочки кредита снижается.


4. Существует зависимость между целями кредита и возвратом его в срок. Наиболее часто просрочка возврата кредита происходит при целях кредита: "операции с автомобилем" и "получение образования". Более надежные те заемщики, которые берут кредит на операции с недвижимостью, затем следуют заемщики с кредитами на проведение свадьбы.
